In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
plot = True

In [ ]:
signals = []

for data_dict_index, data_dict in enumerate(SETTING["model_data_dicts"]):

    print("=" * 8, data_dict["name"], "=" * 8)

    dataframe = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

    dataframe.columns.name = data_dict["axis1_name"]

    dataframe_process = kraft.process_dataframe(
        dataframe, plot=plot, normalization_axis=0, normalization_method="-0-"
    )

    dataframe_process = dataframe_process.loc[
        kraft.select_series_indices(
            dataframe_process.std(axis=1),
            ">",
            standard_deviation=1,
            plot=plot,
            layout={"yaxis": {"title": {"text": "Standard Deviation"}}},
        )
    ]

    kraft.summarize_dataframe(dataframe_process, plot=plot)

    model_signal_type = SETTING["model_signal_types"][data_dict_index]

    if model_signal_type == "raw":

        signal = dataframe_process

    elif model_signal_type == "signed":

        signal_negative = -dataframe_process.clip(upper=0)

        signal_negative.index = signal_negative.index.map(
            lambda index: "(-) {}".format(index)
        )

        signal_positive = dataframe_process.clip(lower=0)

        signal_positive.index = signal_positive.index.map(
            lambda index: "(+) {}".format(index)
        )

        signal = pd.concat((signal_negative, signal_positive))

        signal = signal.loc[(signal != 0).any(axis=1)]

        kraft.summarize_dataframe(signal, plot=plot)

    signals.append(signal)

In [ ]:
model_mode = SETTING["model_mode"]

if model_mode == "range":

    axis_same = 0

    axis_different = 1

elif model_mode == "deep":

    axis_same = 1

    axis_different = 0

signals = kraft.make_dataframes_axis_different(
    kraft.make_dataframes_axis_same(signals, axis_same), axis_different
)

In [ ]:
signal_directory_path = "{}/signal".format(OUTPUT_DIRECTORY_PATH)

kraft.establish_path(signal_directory_path, "directory")

for data_dict_index, data_dict in enumerate(SETTING["model_data_dicts"]):

    signal = kraft.normalize_dataframe(signals[data_dict_index], 1, "0-1")

    signal.to_csv("{}/{}.tsv".format(signal_directory_path, data_dict_index), sep="\t")

    kraft.summarize_dataframe(signal, plot=plot)